<a href="https://colab.research.google.com/github/YiyunLei/NLP-Final-Project/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Load datasets from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import json
import jieba
import re
import codecs
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

In [7]:
# Load cleand and simplified data

def load_data():
  dataset_dir = "/content/drive/MyDrive/NLP_Final_Project/datasets/simplify_json"
  json_files = [dataset_dir + "/train_split.json", dataset_dir + "/valid_split.json"]
  data_files = []
  for json_file in json_files:
      with open(json_file, 'r') as f:
          data_files.append(json.load(f))

  return data_files

In [14]:
# covert big-5 ['Openness', 'Conscientiousness', 'Extraversion','Agreeableness', 'Neuroticism'] to 0:low 1:high
def one_hot_encode_big5(input_data):
  for i in range(len(input_data)):
    for j in range(len(input_data[i])):
        input_data[i][j]['big-5'] = [0 if input_data[i][j]['Openness'] == 'low' else 1,
                                     0 if input_data[i][j]['Conscientiousness'] == 'low' else 1,
                                     0 if input_data[i][j]['Extraversion'] == 'low' else 1,
                                     0 if input_data[i][j]['Agreeableness'] == 'low' else 1,
                                     0 if input_data[i][j]['Neuroticism'] == 'low' else 1]
        del input_data[i][j]['Openness']
        del input_data[i][j]['Conscientiousness']
        del input_data[i][j]['Extraversion']
        del input_data[i][j]['Agreeableness']
        del input_data[i][j]['Neuroticism']
    converted_data = input_data
  return converted_data

In [31]:
# Get (X_train y_train), (X_valid, y_valid) from converted data
def get_X_y(converted_data):
  X_train = []   # name, Utterance
  y_train = []   # big-5
  X_valid = []    # name, Utterance
  y_valid = []    # big-5
  speakers_train, speakers_valid = [],[]
  for i in range(len(converted_data[0])):
    X_train.append((converted_data[0][i]['Speaker'], converted_data[0][i]['Utterance']))
    y_train.append(converted_data[0][i]['big-5'])
    speakers_train.append(converted_data[0][i]['Speaker'])
  speakers_train = list(set(speakers_train))
  print("There are {} speakers in train_data" .format(len(speakers_train)))

  for i in range(len(converted_data[1])):
    X_valid.append((converted_data[1][i]['Speaker'], converted_data[1][i]['Utterance']))
    y_valid.append(converted_data[1][i]['big-5'])
    speakers_valid.append(converted_data[1][i]['Speaker'])
  speakers_valid = list(set(speakers_valid))
  print("There are {} speakers in valid_data" .format(len(speakers_valid)))
  return X_train, y_train, X_valid, y_valid, speakers_train, speakers_valid

In [37]:
# combine same speaker data to one data
def combine_same_speaker_data(speakers, X, y):
    # combine same speaker data to one data and only keep different Utterance and one speaker name
    X_combine = []
    y_combine = []
    for speaker in speakers:
        X_combine.append({speaker:[]})
        y_combine.append({speaker:[]})

    for i in range(len(X)):
        for j in range(len(X_combine)):
            if X[i][0] in X_combine[j]:
                X_combine[j][X[i][0]].append(X[i][1])
                y_combine[j][X[i][0]].append(y[i])

    for i in range(len(y_combine)):
        for key in y_combine[i]:
            y_combine[i][key] = y_combine[i][key][0]
    return X_combine, y_combine

In [16]:
data_files = load_data()
data_files[0][0]
converted_data = one_hot_encode_big5(data_files)
converted_data[0][0]

{'Speaker': '童文洁', 'Utterance': '真巧', 'big-5': [0, 1, 1, 0, 1]}

In [32]:
X_train, y_train, X_valid, y_valid, speakers_train, speakers_valid = get_X_y(converted_data)

There are 220 speakers in train_data
There are 33 speakers in valid_data


In [ ]:
speakers_valid

In [ ]:
X_valid

In [66]:
X_train_combined, y_train_combined = combine_same_speaker_data(speakers_train, X_train, y_train)
X_valid_combined, y_valid_combined = combine_same_speaker_data(speakers_valid, X_valid, y_valid)
X_train_combined[0]

{'艾小天': ['干嘛扣这么紧',
  '像我这样 把衣服往后一撩 把手插在兜里',
  '觉得挺好啊',
  '看咱两了',
  '有钱吗',
  '零钱',
  '来 以水代酒庆祝一下',
  '怎么了',
  '程老师跟你说的一模一样',
  '这么看来好像你们两个更有默契',
  '没说什么',
  '就是说治病救人',
  '还有娱记',
  '其实我觉得你误会他了',
  '他就是不了解病人的情况',
  '让我们查 好通知家属',
  '还有赵冲',
  '他说自己是柯南推理的',
  '就是瞎猜',
  '瞎猜的',
  '到底有没有',
  '听到了',
  '程老师帮帮我',
  '她现在都不骂我了',
  '别说我了',
  '我知道错',
  '赶紧帮我想办法',
  '可是第一',
  '我觉得他们还得谢谢我',
  '要不然他们就要送红包',
  '第二',
  '赵冲的事吧',
  '我还真说不准']}

In [42]:
y_valid_combined[0]

{'叶珊': [0, 1, 0, 0, 1]}

In [65]:
# Use jieba tokenizer
def tokenize_data(speaker, X):
  X_train_tokenized = X
  for speaker in X_train_tokenized:
    for key in speaker:
      for i in range(len(speaker[key])):
          seg_list = jieba.cut(speaker[key][i], cut_all=False)
          speaker[key][i] = " ".join(seg_list)
          if i > 0:
            speaker[key][0] = speaker[key][0] + " " +speaker[key][i]
    for key in speaker:
        for j in reversed(range(1, len(speaker[key]))):
            # delete other key's value
            del speaker[key][j]
    for key in speaker:
        # Remove non-alphanumeric characters (except underscores) and convert to lowercase
        speaker[key][0] = re.sub(r'[^\w\s_]', '', speaker[key][0])
        # Replace multiple whitespace characters with a single space
        speaker[key][0] = re.sub(r'\s+', ' ', speaker[key][0])
  return X_train_tokenized

In [67]:
X_train_tokenized = tokenize_data(speakers_train, X_train_combined)
X_valid_tokenized = tokenize_data(speakers_valid, X_valid_combined)
X_train_tokenized[0]

{'艾小天': ['干嘛 扣 这么 紧 像 我 这样 把 衣服 往后 一 撩 把手 插 在 兜里 觉得 挺 好 啊 看 咱 两 了 有钱 吗 零钱 来 以水代酒 庆祝 一下 怎么 了 程 老师 跟 你 说 的 一模一样 这么 看来 好像 你们 两个 更 有 默契 没说 什么 就是说 治病救人 还有 娱记 其实 我 觉得 你 误会 他 了 他 就是 不 了解 病人 的 情况 让 我们 查 好 通知 家属 还有 赵冲 他 说 自己 是 柯南 推理 的 就是 瞎猜 瞎猜 的 到底 有没有 听到 了 程 老师 帮帮我 她 现在 都 不 骂 我 了 别说 我 了 我 知道 错 赶紧 帮 我 想 办法 可是 第一 我 觉得 他们 还 得 谢谢 我 要不然 他们 就要 送 红包 第二 赵冲 的 事 吧 我 还 真 说 不准']}

In [78]:
y_valid

[[0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [1, 1, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [0, 1, 1, 1, 1],
 [1, 1, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [1, 1, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [1, 1, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [1, 1, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 0],
 [0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1],
 [0, 1, 1,

In [83]:
# Remove speaker from X and get the value of y(O of big-5)
def clean_X_y(X_tokenized, y):
  X_train_cleaned = []
  y_train_cleaned = []
  for speaker in X_tokenized:
          X_train_cleaned.append(speaker)
  for row in y:
      y_train_cleaned.append(row[0])
    
  return X_train_cleaned, y_train_cleaned

In [86]:
X_train_cleaned, y_train_cleaned = clean_X_y(X_train_tokenized, y_train)
X_valid_cleaned, y_valid_cleaned = clean_X_y(X_valid_tokenized, y_valid)
X_train_cleaned[0]

{'艾小天': ['干嘛 扣 这么 紧 像 我 这样 把 衣服 往后 一 撩 把手 插 在 兜里 觉得 挺 好 啊 看 咱 两 了 有钱 吗 零钱 来 以水代酒 庆祝 一下 怎么 了 程 老师 跟 你 说 的 一模一样 这么 看来 好像 你们 两个 更 有 默契 没说 什么 就是说 治病救人 还有 娱记 其实 我 觉得 你 误会 他 了 他 就是 不 了解 病人 的 情况 让 我们 查 好 通知 家属 还有 赵冲 他 说 自己 是 柯南 推理 的 就是 瞎猜 瞎猜 的 到底 有没有 听到 了 程 老师 帮帮我 她 现在 都 不 骂 我 了 别说 我 了 我 知道 错 赶紧 帮 我 想 办法 可是 第一 我 觉得 他们 还 得 谢谢 我 要不然 他们 就要 送 红包 第二 赵冲 的 事 吧 我 还 真 说 不准']}